## Модель

Как мы с вами обсудили на лекции -- одним из основных блоков программы для обучения нейронных сетей является *описание модели*.

Что такое модель? Вне зависимости от используемого фреймворка, это блок кода, который показывает нашей программе как исходные признаки должны быть преобразованы в предсказания модели. 

А вот само определение модели зависит от фреймворка и каждый из них предлагает свою структуру (правила) описания моделей (которые сейчас становятся все более похожи друг на друга). Структура необходима для того, чтобы фреймворк мог построить вычислительный граф и мог правильно считать его производные.

## Keras API для Tensorflow

Изучение Tensorflow мы начнем сразу с высокоуровневого интерфейса Keras, который считается основным интерфейсом для реализации нейронных сетей в tensorflow и с недавних пор является его частью. Теоретически, все, что мы будем делаем можно реализовать без использования Keras, но в таком случае нам придется писать и тестировать большое количество самописных классов и функций, что не является целью данного блока


## Модель в Keras

 Модель в Keras -- это объект класса **tf.keras.Model**

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
model = tf.keras.Model()
type(model)

In [ ]:
tf.keras.Model?
"`Model` groups layers into an object with training and inference features."

Из документации видно, что модель -- работает со слоями. Что же такое слой? Если смотреть на модель как на конструктор, то слой -- это кубик.

Другими словами слой -- это слой нейронной сети. Их типов существует [очень много](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Но на данный момент мы знаем только один слой -- полносвязный. Давайте посмотрим, как его определить в Keras.


### Полносвязный слой
Полносвязный слой в Tensorflow называется Dense и живет в tensorflow.keras.layers.

In [ ]:
from tensorflow.keras.layers import Dense
import numpy as np
#Dense?

Он имеет следующие параметры (показаны лишь самые основные):

```
Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', ...)
```


*   units -- количество нейронов на выходе из слоя
*   activation -- функция активации, например "sigmoid", "relu", "softmax". по умолчанию -- слой без активации
*   use_bias -- применять ли bias 
*   kernel_initializer и bias_initializer -- правила инициализации матрицы весов (kernel) и баеса. Это часто используемые значения "по-умолчанию". Скорее всего вам не придется их никогда менять)

Обратите внимание в этом списке нет "количества нейронов на входе". В keras необходимо указывать количество входных нейронов только в первом слое сети. В остальных он посчитает самостоятельно -- ведь, количество входных нейронов в слой, должно быть равно количеству выходных нейронов на предыдущем слое. 

В первом слое размер входа указывается через input_shape. 

Рассмотрим пример ниже. В нем мы “применим” слой к инпуту x. Для применения слоя нужно вызвать его метод `__call__` , который вызывается когда аргумент передается в скобках, это мы делаем в третьей строчке.


In [ ]:
x = np.ones((4, 3)) # 4 объекта, 3 признака

dense_layer = Dense(units=2, input_shape=(3,))
output = dense_layer(x)
print(f"Output: {output}")

In [ ]:
w, b = dense_layer.get_weights() 
print(f"Weights: {w}")
print(f"Bias: {b}")

In [ ]:
manual_output = np.matmul(x, w) + b
print(f"Manual output: {manual_output}")

In [ ]:
x = np.ones((4, 3))
dense_layer = Dense(2, input_shape=(3,), use_bias=False) # можно инициализировать слой без баеса
output = dense_layer(x)
print(dense_layer.get_weights())

## Sequential Model
А как же нам "прицепить" один слой к другому? Самый простой способ это сделать -- класс Sequential. Он может быть использован в подавляющем большинстве случаев. Как следует из названия -- он последовательно применяет один слой за другим.

In [ ]:
model = tf.keras.Sequential()
model.add(Dense(10, input_shape=(10,), activation="relu")) # скрытый слой 1
model.add(Dense(20, activation="relu", 
                bias_initializer=tf.keras.initializers.Constant(2.0))) # скрытый слой 2, с не дефолтной инициализацией
model.add(Dense(5, activation="softmax")) # выходной слой

In [ ]:
sample_input = np.random.rand(2, 10)
output = model(sample_input)

print(output)

In [ ]:
print(output.numpy().sum(1)) # на выходе -- софтмакс, значит сумма == 1

In [ ]:
# проверим как инициализирован второй скрытый слой 
# для этого обратимся к нему с помощью:
model.layers[1].get_weights()[1]

Есть очень удобная функция, которая подскажет сколько в нашей модели параметров, и какие размерности получены на промежуточных слоях -- model.summary()

In [ ]:
model.summary() # None обозначает, что размер батча может быть произвольный

In [ ]:
# если не указать размер инпута в первом слое, то summary не сработает
model_no_input = tf.keras.Sequential()
model_no_input.add(Dense(10, activation="relu")) # скрытый слой 1 
model_no_input.add(Dense(20, activation="relu"))
model_no_input.add(Dense(5, activation="softmax")) # выходной слой

model_no_input.summary() # None обозначает, что размер батча может быть произвольный

Или для еще более наглядного представления -- tf.keras.utils.plot_model()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

## Functional API
Но с помощью Sequential можно представить ограниченное (хоть и часто достаточное на практике) множество моделей. Представьте, что вам пригодилось реализовать такую модель:

<img src="https://drive.google.com/uc?export=view&id=1-1Bltu3VDEDbTl0liRmfs5Dr3r5NzBCR" width=600>

Реализовать такую модель с помощью Sequential нельзя. Но здесь к нам на помощь приходит Functional API.

Для понимания Functional API нужно начать относится к сети как к графу. В таком случае вершины -- слои, а ребра связывают выход одного слоя с входом другого. Иллюстрация выше как раз поможет это сделать.

Определять слои в коде мы уже умеем, теперь нужно научиться "рисовать стрелки". Для того чтобы это сделать нужно просто применить один слой к выходу другого! Рассмотрим пример ниже.

In [ ]:
# cпециальный Инпут слой, показывает keras куда "положить" данные при использовании модели
input_layer = tf.keras.layers.Input(shape=(10, ), name="Input") 
l1 = Dense(10, name="Layer1") # определили первый слой
l1_output = l1(input_layer) # соеденили его с инпутом, "нарисовали" самую правую стрелку на картинке выше

l2_output = Dense(10, name="Layer2")(input_layer) # дальше будем определять чуть компактнее
l3_output = Dense(10, name="Layer3")(l2_output)
l4_output = Dense(10, name="Layer4")(l3_output)

l5_output = Dense(10, name="Layer5")(l2_output)

# специальный слой, который сложит выходы всех слоев, которые ему передали
sum_output = tf.keras.layers.Add(name="SumLayer")([l1_output, l4_output, l5_output])

# выходной слой
output = Dense(10, name="Output")(sum_output)

# создаем модель. нужно показать что мы считаем входом в нашу модель, а что выходом.
model = tf.keras.Model(inputs=input_layer, outputs=output)

tf.keras.utils.plot_model(model, show_shapes=False)

In [ ]:
model.summary()

In [ ]:
# если мы укажем в аутпутах слой, для вычисления которого не нужно было считать весь граф -- 
# keras автоматически обрежет граф. 
model = tf.keras.Model(inputs=input_layer, outputs=l4_output)

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# можно указать несколько выходов
model = tf.keras.Model(inputs=input_layer, outputs=[l4_output, l5_output, l1_output])

tf.keras.utils.plot_model(model, show_shapes=False)

## Заключение

Мы познакомились с одним из самых главных объектов в Keras -- модель. И сделали первый шаг -- научились ее определять. 

Теперь вам не составит труда построить модель самостоятельно по ее описанию. Именно это вы и попробуете сделать на [практике](https://colab.research.google.com/drive/1OFKvrSYHU71O68uXcunQtTFJE4sLZW6E). 

А как только вы почувствуете себя уверенно, мы перейдем к самому интересному -- обучению модели.

